
## Search through StructureDefinition properties

*USE Python 3.7+ to maintain order of Json files*

This script requires several modules listed below and SUSHI (see how to install and use [here](https://fshschool.org/docs/sushi/))
The URLS are fetched from a simple external csv file which I create from the output of the github_search.ipynb script

- Fetch URLs from csv file
- Fetch published URL
- Convert to Python fhir-obj
- Filter for StructureDefinition files
- Search for SD elements

### import python modules

In [9]:
from pathlib import Path
import pandas as pd
import csv
from json import loads, dumps
from requests import get,post
from fhir.resources.structuredefinition import StructureDefinition
from pprint import pprint

headers = {'Accept': 'application/fhir+json', 'Content-Type': 'application/fhir+json'}
in_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/my-notes/MedReq-MedDisp-profiles.csv')
in_path = Path(r'/Users/ehaas/Documents/FHIR/US-Core/my-notes/SD_URL.md.csv')
fsh_project = '/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/test_files/my-project'
sushi_config_path = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/test_files/my-project/sushi-config.yml')
fsh_file_path = Path(r'/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/test_files/my-project/input/fsh/file.fsh')
fsh_gen_resource_path = Path('/Users/ehaas/Documents/Python/Jupyter/MyNotebooks/utils/test_files/my-project/fsh-generated/resources')

in_path

PosixPath('/Users/ehaas/Documents/FHIR/US-Core/my-notes/SD_URL.md.csv')

### Loop through SDs
 - read sheet as DF
 - iterate through urls
   - fetch SD as xml, json, or fsh
   - convert all to python fhir object  ( fsh files require runing new sushi project each time )
   - filter out the SD resources for the type interested in  ( `test_type` )
   - look for the particular element of interest. `test_me`
   - print out the stats of how many found and the canonicals ( or best guess canonical for sushi files )

In [10]:
test_type = 'Coverage' #'MedicationRequest' #'ServiceRequest'#Coverage
#test_me = ["ServiceRequest.reasonCode","ServiceRequest.reasonReference",] #["MedicationRequest.reasonCode","MedicationRequest.reasonReference",]#["ServiceRequest.reasonCode","ServiceRequest.reasonReference",]
test_me =["Coverage.period",]
count1 = 1
count2 = 1
list1 = []
list2 = []

def check_element(sd):
    global count1, count2, list1, list2
    if sd.type == test_type:
        print(f'({count1}) Type = {test_type}')
        count1 += 1
        list1.append(sd.url)
        print(f'sd.id = {sd.id}')
        element_paths = [e.path for e in sd.differential.element]
        if any(item in element_paths for item in test_me):
                print(f'>>>>>>>>>({count2}) reason[x] element found!!<<<<<<<<<<')
                list2.append(sd.url)
                count2 += 1

                
def update_sushi(my_file):
    '''update the sushi-config.yaml file'''
    project_code  = my_file.split('/')[4:5][0]
    print(f'project_code: {project_code}')
    sushi_config = f'''
FSHOnly: true
fhirVersion: 4.0.1
canonical: "http://hl7.org/fhir/us/{project_code}"
name: Temp
status: draft
''' 
    # print(sushi_config)
    # print(config_path)
    sushi_config_path.write_text(sushi_config)

def make_fhir_object():
    for file in fsh_gen_resource_path.glob('*.json'):
        print(file)
        try:
            sd = StructureDefinition.parse_file(file)
            file.unlink()
        except:
            print(f'Error parsing {my_file}:\n {response.text[:75]}')
            file.unlink()
        else:
            return sd


df = pd.read_csv(in_path, header=0, encoding='utf-8')
for i,my_file in enumerate(df.url):
    print(f'*************************************\n{i+1}: {my_file}\n*************************************')
    with get(my_file, headers=headers) as response:
        if response.status_code == 200:
            if my_file.split('.')[-1] == 'xml':
                data = response.text.replace('<?xml version="1.0" encoding="UTF-8"?>','',1) # check all spellings!!!
                data = data.replace('<?xml version="1.0" encoding="utf-8"?>','',1) # check all spellings!!!
                try:       
                    sd = StructureDefinition.parse_raw(data, content_type="text/xml")
                except:
                    print(f'Error parsing {my_file}:\n {response.text[:75]}')
                else:
                    check_element(sd)
            if my_file.split('.')[-1]  == 'json':
                try:
                    sd = StructureDefinition.parse_obj(response.json())
                except:
                    print(f'Error parsing {my_file}:\n {response.text[:75]}')
                else:
                    check_element(sd)
            if my_file.split('.')[-1]  == 'fsh':
                print('convert fsh to json')
                update_sushi(my_file)
                fsh_file_path.write_text(response.text)
                #build json from fsh files using sushi
                !sushi $fsh_project > "/Users/ehaas/Documents/Python/Jupyter/sushi.txt"
                sd = make_fhir_object()
                try:
                    check_element(sd)
                except AttributeError:
                    print(f'Error parsing {my_file}:\n sd = {sd}')

print(f'Total {test_type} Profiles = {len(set(list1))}')
pprint(set(list1))
print(f'Total {test_type} Profiles with {test_me} in differential = {len(set(list2))}')
pprint(set(list2))
print(f'Total Files = {i+1}')
                

*************************************
1: https://raw.githubusercontent.com/HL7/dme-orders/b11211e5aa26ee2e1343f13e0f3cc088436026cf/input/resources/structuredefinition/PAOX-coverage.xml
*************************************
(1) Type = Coverage
sd.id = PAOX-coverage
*************************************
2: https://raw.githubusercontent.com/HL7/davinci-deqm/168acbe28688e4ea9641a57389017be77c4a3da7/input/resources/StructureDefinition-coverage-deqm.xml
*************************************
(2) Type = Coverage
sd.id = coverage-deqm
*************************************
3: https://raw.githubusercontent.com/HL7/davinci-pcde/58ace3091511bbfe7bc125a71485f07bfff972ac/input/resources/structuredefinition-pcde-task-request.xml
*************************************
*************************************
4: https://raw.githubusercontent.com/HL7/davinci-alerts/c006d6804eb706609361fa778f303da040530c10/input/resources/drafts/StructureDefinition-admit-discharge-notification-coverage.xml
*******************